##### 7.5 CNN 구현

- 손글씨 숫자를 인식하는 CNN 구현  
- SimpleConvNet: `Convolution-ReLU-Polling-Affine-ReLU Affine-Softmax'

- 초기화 때 인수
    - input_dim: 입력 데이터(채널 수, 높이, 너비)의 차원 

    - 'conv_param': 합성곱 계층의 하이퍼파리미터(딕셔너리)  
        - 딕셔너리의 키  
            - filter_num(필터 수), filter_size(필터 크기)  
            - stride(스트라이드), pad(패딩)  
            - hidden_size(은닉층(완전연결)의 뉴런 수), output_size(출력층(완전연결)의 뉴런 수)  
            - weight_init_std(초기화 때의 가중치 표준편차)

In [3]:
import numpy as np

class SimpleConvNet:        # 다시 코드 체크해볼 것
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride'=1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']

        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2 ) * (conv_output_size /2))


        self.params = {}

        # 1번째 층의 합성곱 계층의 가중치와 편향
        self.params['W1'] = weight_init_std * np.random.rand(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)

        # 나머지 두 완전연결 계층의 가중치와 편향
        self.params['W2'] = weight_init_std * np.random.rand(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)

        self.params['W3'] = weight_init_std * np.random.rand(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)


        self.layers = OrderedDict()     # 순서가 있는 딕셔너리 layers에 계층을 차례로 추가

        self.layers['Conv1'] = Convolution(self.params['W1'], self.params[b1], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w = 2, stride = 2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])

        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    
    # x: 입력 데이터, t: 정답 레이블

    def predict(self, x):       # layers에 추가한 계층을 맨 앞에서부터 차례로 forward 호출해 결과를 다음 계층에 전달
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):       # predict 메서드의 결과를 인수로 마지막 층의 forward 메서드 호출 == 첫 계층부터 마지막 계층까지 forward 처리
        y=self.predict(x)

        return self.lastLayer.forward(y, t)

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)


        # 역전파-> 매개변수의 기울기 연산
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)


        # 결과 저장
        grads = {}

        grads['W1'] = self.layerrs['Conv1'].dW
        grads['b1'] = self.layerrs['Conv1'].db

        grads['W2'] = self.layerrs['Affine1'].dW
        grads['b2'] = self.layerrs['Affine1'].db

        grads['W3'] = self.layerrs['Affine2'].dW
        grads['b3'] = self.layerrs['Affine2'].db

        return grads    # 가중치 매개변수의 기울기 저장


SyntaxError: invalid syntax (3751956128.py, line 4)

- SimpleConvNet를 MNIST 데이터셋으로 학습 시,  
    - 훈련 데이터에 대한 정확도: 99.82%
    - 시험 데이터에 대한 정확도: 98.96%